#Scraps all auctions from a specific auction house#

In [1]:
from BeautifulSoup import BeautifulSoup
import urllib,string,csv,sys,os
import pandas as pd

In [2]:
# Set the fixed parts of the URL
fx_url_s = 'http://en.artron.net/auction/paimai-result.php?page='
fx_url_e = '&jgid=JG1059'

In [78]:
# Load page 1 to get total page count and url row indice
datas = urllib.urlopen('http://en.artron.net/auction/paimai-result.php?page=1&jgid=JG1059').read()
s = BeautifulSoup(datas)
tables = s.findAll("table")
# total page count
page = tables[2].findAll('td')[4].string.strip()
page_count = [int(s) for s in page.split() if s.isdigit()][0]
# url rows indice
count = []
for i in range(1,96):
    if len(tables[4].findAll('td')[i].findAll('a',href=True))>0:
        count.append(i)

In [40]:
datas = urllib.urlopen('http://en.artron.net/auction/paimai-result.php?page=1&jgid=JG1059').read()
s = BeautifulSoup(datas)
tables = s.findAll("table")
len(tables[4].findAll('td'))


96

In [80]:
def get_auctions(url_s,url_e,total_page):
    url_pre = 'http://en.artron.net/auction/'
    urls = []
    # Read in page by page
    for p in range(1,total_page+1):
        url = url_s+str(p)+url_e
        data = urllib.urlopen(url).read()
        soup = BeautifulSoup(data)
        
        try:
            tables = soup.findAll("table")
        except AttributeError as e:
            raise ValueError("No valid table found")
        for i in count:
            auction_href = tables[4].findAll('td')[i].findAll('a',href=True)[0]['href']
            urls.append(url_pre+auction_href)
        return urls

In [82]:
auctions = get_auctions(fx_url_s,fx_url_e,page_count)

In [86]:
auctions[:] = [url for url in auctions if ('paimai-list' in url)]

In [89]:
len(auctions)

20

In [90]:
auctions[:5]

[u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029244',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029245',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2029247',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2027078',
 u'http://en.artron.net/auction/paimai-list.php?zcid=PZ2027079']